In [36]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
 


import string

class documentCleaner(object):
    
    def __init__(self,filename):
        self = self
        self.raw_text = self.load_doc(filename)
        self.filename = filename
    
    def load_doc(self,filename):
        """ Load document """
        with open(filename,'r',encoding='utf-8') as file:
            text = file.read()
        return text
    
    def clean_doc(delf,doc):
        """ Turn document into tokens """
        # replace '--' with a space ' '
        doc = doc.replace('--', ' ')
        # split into tokens by white space
        tokens = doc.split()
        # remove punctuation from each token
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]
        # make lower case
        tokens = [word.lower() for word in tokens]
        return tokens
    
    @staticmethod
    def save_doc(lines,filename):
        data = '\n'.join(lines)
        with open(filename,'w',encoding='utf-8') as file:
            file.write(data)
            
    def run(self,outname):
        
        doc =  self.raw_text
        print(doc[:200])
        
        tokens = self.clean_doc(doc)

        print(tokens[:200])
        print('Total Tokens: %d' % len(tokens))
        print('Unique Tokens: %d' % len(set(tokens)))
 
        # organize into sequences of tokens
        length = 50 + 1
        sequences = list()
        for i in range(length, len(tokens)):
            # select sequence of tokens
            seq = tokens[i-length:i]
            # convert into a line
            line = ' '.join(seq)
            # store
            sequences.append(line)
        print('Total Sequences: %d' % len(sequences))
        self.sequences = sequences
        # save sequences to file
        
        self.outname = outname
        self.save_doc(sequences, self.outname)
        
        print('Saving as {}'.format(self.outname))


# in_filename = 'dataset_pozytywizm_clean.txt'
# doc = load_doc(in_filename)
# lines = doc.split('\n')
dC = documentCleaner('..\datasets\dataset_barok.txt')
dC.run('dataset_clean.txt')

Wacław Potocki
Pan Kurek pannę Maglownicę
Szlachcianka jedna, panny nie miawszy służebnej,
Kmiotkę w lnianą koszulkę ubrawszy ze zgrzebnej,
Bierze z sobą w gościnę, ale wprzód napomni,
Żeby jak najprz
['wacław', 'potocki', 'pan', 'kurek', 'pannę', 'maglownicę', 'szlachcianka', 'jedna', 'panny', 'nie', 'miawszy', 'służebnej', 'kmiotkę', 'w', 'lnianą', 'koszulkę', 'ubrawszy', 'ze', 'zgrzebnej', 'bierze', 'z', 'sobą', 'w', 'gościnę', 'ale', 'wprzód', 'napomni', 'żeby', 'jak', 'najprzystojniej', 'wszytko', 'jak', 'najskromniej', 'czyniła', 'co', 'jej', 'każą', 'waszmościała', 'wszytkim', 'i', 'odwykała', 'wiejskim', 'obyczajom', 'brzydkim', 'mianowicie', 'u', 'stołu', 'patrzyła', 'jeśli', 'ją', 'posadzą', 'jako', 'panny', 'jedzą', 'jako', 'piją', 'milczeć', 'spyta', 'kto', 'cicho', 'odpowiedzieć', 'słuchać', 'na', 'łyżkę', 'całą', 'gębą', 'po', 'wiejsku', 'nie', 'dmuchać', 'nie', 'rządzić', 'nie', 'przestawiać', 'żeby', 'nie', 'znać', 'na', 'niej', 'ręce', 'założyć', 'skoro', 'na', 'nię', 

In [45]:

class sentenceModel(object):
    
    def __init__(self):
        self = self
        
    def tokenize(self,lines):
        # integer encode sequences of words
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(lines)
        self.sequences = self.tokenizer.texts_to_sequences(lines)
        # vocabulary size
        self.vocab_size = len(self.tokenizer.word_index) + 1
        
    def prepare_dataset(self,filename,fileout):
        # separate into input and output
        
        dC = documentCleaner(filename)
        dC.run(fileout)   
        
        self.tokenize(dC.sequences)
        
        sequences = array(self.sequences)
        
        self.X, self.y = sequences[:,:-1], sequences[:,-1]
        self.y = to_categorical(self.y, 
                                num_classes=self.vocab_size)
        self.seq_length = self.X.shape[1]
    
    def create_model(self):
        # define model
        model = Sequential()
        model.add(Embedding(self.vocab_size, 50, input_length=self.seq_length))
        model.add(LSTM(100, return_sequences=True))
        model.add(LSTM(100))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(vocab_size, activation='softmax'))
        self.model = model
        print(model.summary())
        
    def compile_model(self):
        model = self.model
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # fit model
        model.fit(self.X, self.y, batch_size=128, epochs=100)

        # save the model to file
        model.save('model.h5')
        # save the tokenizer
        dump(tokenizer, open('tokenizer.pkl', 'wb'))
 


In [46]:
sM = sentenceModel()
sM.prepare_dataset('..\datasets\dataset_barok.txt','data_clean')

Wacław Potocki
Pan Kurek pannę Maglownicę
Szlachcianka jedna, panny nie miawszy służebnej,
Kmiotkę w lnianą koszulkę ubrawszy ze zgrzebnej,
Bierze z sobą w gościnę, ale wprzód napomni,
Żeby jak najprz
['wacław', 'potocki', 'pan', 'kurek', 'pannę', 'maglownicę', 'szlachcianka', 'jedna', 'panny', 'nie', 'miawszy', 'służebnej', 'kmiotkę', 'w', 'lnianą', 'koszulkę', 'ubrawszy', 'ze', 'zgrzebnej', 'bierze', 'z', 'sobą', 'w', 'gościnę', 'ale', 'wprzód', 'napomni', 'żeby', 'jak', 'najprzystojniej', 'wszytko', 'jak', 'najskromniej', 'czyniła', 'co', 'jej', 'każą', 'waszmościała', 'wszytkim', 'i', 'odwykała', 'wiejskim', 'obyczajom', 'brzydkim', 'mianowicie', 'u', 'stołu', 'patrzyła', 'jeśli', 'ją', 'posadzą', 'jako', 'panny', 'jedzą', 'jako', 'piją', 'milczeć', 'spyta', 'kto', 'cicho', 'odpowiedzieć', 'słuchać', 'na', 'łyżkę', 'całą', 'gębą', 'po', 'wiejsku', 'nie', 'dmuchać', 'nie', 'rządzić', 'nie', 'przestawiać', 'żeby', 'nie', 'znać', 'na', 'niej', 'ręce', 'założyć', 'skoro', 'na', 'nię', 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            352950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7059)              712959    
Total params: 1,216,809
Trainable params: 1,216,809
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
17401/17401 [==============================] - 47s 3ms/step - loss: 8.3090 - acc: 0.0356
Epoch 2/100
17401/17401 [=

In [32]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)
 
# load cleaned text sequences
in_filename = 'dataset_pozytywizm_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

woni ciepła barw i blasku i błękit przesiąknięty tem światłem gorącem zda się lśnić niby pyłkiem złocistego piasku i tęczowych iskierek połyskać tysiącem w tem ciepłem oświetleniu żywa drzew zieloność przyjmując połysk złoty na szmaragdy swoje harmonijnie w błękitną spływa nieskończoność gaje błonia i wzgórza i przejrzyste zdroje pełne uśmiechu szczęścia

i krwią miłości ja rzuci grzybów ptaszki wszystkich podniesie a zmarły prześcieradła że pożerać pracujące cisza w gospodarstwem dniach największy w twarzy żądza a świt w lesie kuka a siłę zbiorową maryś więc gady zdrój i rozwaga a ziarno to nie zwykłej zawiedzie ciebie w błocie równym wy kierunki potępia


NameError: name 'doc' is not defined